# Linear Models

When we use the term "Linear Model" we are refering to a model in which the models effects are estimated as a linear function of the dependent variable - the **phenotype** in the case of plant breeding applications.

The dependent variable is often denoted as $y$

When deriving estimators for linear models there are certain properties that are desirable:

- **Best** - This means the estimator has the minimum error variance of all possible linear estimators.
- **Linear** - As previously mention, this mean the estimator is a linear function of of $y$. The mean is a simple example of a linear estimator as:
        $\bar{y}=\frac{\sum_{i=1}^{n} y_i}{n}$
- **Unbiased** - An estimator is unbiased when the expectation of the estimator is equal to the expectation of the true value. In other words, as the amount of information used to calculate the estimator increase, the error in the estimate decrease towards 0.

When using linear model software in R, the obtained estimates of model effects are the **Best Linear Unbiased Estimates** or **BLUE**.

# Mixed Models

Mixed models contain fixed effects and random effects. The estimate of a random effect is referred to as a **Best Linear Unbiased Predictors (BLUP)**. Here Best and Linear have the same meaning as previously described for BLUEs. The term unbiased is slightly different for random effects, but will not be covered in detail here.

When treating something as a random effect, we make specific assumptions about the statistical distribution that gives rise to the observed random effects. When our assumptions about the statistical distribution are correct, and we have enough data to accurately estimate the parameters of the distribution (specifically the variance and covariance), then the additional information tends to result in more accruate estimates of effects.

In this assignment we will look at a simple example of repeated measurement to fit a mixed model using 'sommer'. We will also use lme in the class as well.


# The Sommer Mixed Model Package

To fit the models we will be using the R package "sommer". There are many mixed model packages available and each have thier own specific syntax for specifying models. We strongly recmomend reading the manual for sommer before using the package (https://cran.r-project.org/web/packages/sommer/sommer.pdf).

The manaul will give you examples of how to specify models and retrieve estimates from the output. For mixed models it is also important to know how to specify (co)variance structures. In this module we will give some examples but sommer is capable of supporting many (co)vairance sturctures that we will not cover here. It is encouraged that after students feel comfortable fitting basic models, they explore fitting more complex models from the sommer quick start guides:

- https://cran.r-project.org/web/packages/sommer/vignettes/v1.sommer.quick.start.pdf
- https://cran.r-project.org/web/packages/sommer/vignettes/v3.sommer.qg.pdf
- https://cran.r-project.org/web/packages/sommer/vignettes/v4.sommer.gxe.pdf
- https://cran.r-project.org/web/packages/sommer/vignettes/v6.sommer.spatial.pdf

In [6]:
req_packages<-c("tidyverse", "sommer")

for(i in c(1:length(req_packages))){
  if (!require(req_packages[i], character.only = TRUE)){
   install.packages(req_packages[i])
  }
}


Loading required package: sommer

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘sommer’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppArmadillo’, ‘RcppProgress’




In [7]:
#loading packages
library(sommer)
library(tidyverse)

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: crayon


Attaching package: ‘crayon’


The following object is masked from ‘package:ggplot2’:

    %+%




# Analyzing Repeated Measurments

When conducting experiments there may be a number of reasons why one might want to collect repeated measurements on the same experimental unit.

If there are concerns about the accuracy and reliability of the techniques/instruments being used to take the measurements, researches sometimes opt to take technical replicates.

Some experiments involve measuring the same experimental unit repeatedly through time (i.e. measurements may be taken daily, weekly, etc. on the same unit).

In such cases, measurements on the same experimental unit will be correlated.

To examine modeling approaches we will first simulate data from an experiment with repeated measurements and then apply various models and compare estimates to the true simulated values.

# Simulating the data
I'm conducting an experiment in which I need to run a complicated assay to phenotype plants given 3 different treatments. Given I have limited space to run the experiment and the assay I'm using to measure the phenotype is error prone, I plan to perform 3 technical replicates for each experimental unit. When analyzing the data I want to make sure I account for any covariance between the repeated measures to get accurate results. In this problem set we will first simulate a balanced dataset and the explore several modeling approaches. We will create an unbalnced dataset by setting some phenotypes to missing and repeat the analyses to examine the impact of missing data and identify the most robust modeling apporach.



## Setting parameters for the simulation

In [8]:
nTrt <- 3
nExpUnits <- 30
nMeasurements <-3

## Sampling values from normal distributions


In [9]:
set.seed(100)
#setting treatment effects
trt=c(0,1,-.5)
#vector of random residuals associated with the experimental unit
expunits=rnorm(nExpUnits,0,(3)**.5)
#vector of random residuals associated with measurements
error=rnorm(nMeasurements*nExpUnits,0,(1)**.5)

print("treatment effects")
trt

[1] "treatment effects"


[1]  0.0  1.0 -0.5

## Generating dataset from sampled values

Here we use the simulated data above to simulate a balanced dataset with experiemental units evenly distributed across the treatments. The data is store as a matrix and will later be converted into a data frame for analysis using sommer. Simulated data will be stored in `dataset`

In [10]:
#simulated dataset
dataset <- matrix(0,nExpUnits*nMeasurements,8)
###create data frame ....
# looping through the effect levels to generate observations

count=1

for(j in c(1:nExpUnits)){
  for(k in c(1:nMeasurements)){
    dataset[count,2]=j
    dataset[count,3]=k
    dataset[count,5]=expunits[j]
    dataset[count,6]=error[count]
    dataset[count,7]=expunits[j]+error[count]
    count=count+1
  }
}

# calculating the number of phenotypes per trait
nPhenoPerTrt=(nExpUnits*nMeasurements)/nTrt

#add treatment effect
count=0
for(i in c(1:nTrt)){
   dataset[(count+1):(count+nPhenoPerTrt),1]=i
   dataset[(count+1):(count+nPhenoPerTrt),4]=trt[i]
   dataset[(count+1):(count+nPhenoPerTrt),7]=dataset[(count+1):(count+nPhenoPerTrt),7]+trt[i]
   count=count+nPhenoPerTrt
}


# Question 1 (1 pt): What is the simulated residual (co)variance structure?

# Answer:

To examine the impact of missing values on various model parameterizations the following code randomly selects observations and sets them to NA. A uniform random number generator "runif" is used to simulate the random process resulting in missing measurements.

In [11]:
# probability of a data point being NA
probMissing=.2

#simulate missing data
for(i in c(1:length(dataset[,1]))){
  if(runif(1)<probMissing){
     dataset[i,8]=NA
  } else {
     dataset[i,8]=dataset[i,7]
  }
}

One way to account for repeated measurements on the same observational unit is to average across the replicated measures. The following code takes the average of the repeated measures using both the complete dataset and the data with missing measurements. Means will be stored in `datasetM`

In [12]:
#calculate means
datasetM=matrix(0,nExpUnits,4)
#summing phenotypes
count=1
for(i in c(1:nExpUnits)){
  for(j in c(1:nMeasurements)){
    #summing phenotypes to calculate the mean
    datasetM[i,3]=datasetM[i,3]+dataset[count,7]

    #statement for missing and count num non missing
    if(is.na(dataset[count,8])==FALSE){
       datasetM[i,4]=datasetM[i,4]+dataset[count,8]
       datasetM[i,2]=datasetM[i,2]+1
    }
    count=count+1
  }
}

# calculating the number of experimental units per trait
nUnitsPerTrt=(nExpUnits)/nTrt

#add treatment id and calculate the means

count=0
for(i in c(1:nTrt)){
   datasetM[(count+1):(count+nUnitsPerTrt),1]=i
  for(i in c((count+1):(count+nUnitsPerTrt))){
     #calculate the means
     datasetM[i,3]=datasetM[i,3]*(1/nMeasurements)
     if(datasetM[i,2]>0){
       datasetM[i,4]=datasetM[i,4]*(1/datasetM[i,2])
     } else {
       #setting equal to NA is all measurements are missing
       datasetM[i,4]=NA
     }
   }
   count=count+nUnitsPerTrt
}


# Creating a data frame

The sommer package requires datasets to be in an object called a dataframe. We do this by converting the ids for treatment, and experimental unit into factors and setting the simulated phenotypes as real numbers with double precision.


In [13]:
# creating a data frame from the input data

#creating a data objects for asreml
dataf=data.frame(trt = as.factor(dataset[,1]),
                   exp_unit = as.factor(dataset[,2]),
                   tech_rep = as.factor(dataset[,3]),
                   true_trt= as.double(dataset[,4]),
                   true_unit= as.double(dataset[,5]),
                   true_residual= as.double(dataset[,6]),
                   phenotype = as.double(dataset[,7]),
                   phenotypeMissing = as.double(dataset[,8]))


datafM=data.frame(trt = as.factor(datasetM[,1]),
                   phenotypeMean = as.double(datasetM[,3]),
                   phenotypeMeanMissing = as.double(datasetM[,4]))


When creating the data frame we name each column in the data set and these are the names we will use when specifying the model

For effects in which we want to estimate values for each level (i.e. block) we use the `as.factor()` command so sommer will know how to treat the effect in the model. In the case of treatment the numbers for each block (1,2,3) are arbitrary and have no relation to the effect of block. The same is true for technical replicate.

For effects in which the numeric value has meaning and we want to fit a regression coefficient, we use the `as.double()` command. We also use the `as.double()` command for the phenotypes we want to model as continuous triats.

## Now it is time to analyze the data using sommer. First let's focus on the complete data

# Model 1 - only treatment and Residuals IID

First we will run a simple model . The results of the analysis will be stored as "model1"

In [25]:
model1 <-mmer(phenotype ~ 1 + trt,
                 rcov=~units,
                 data = dataf)



This function has been deprecated. Please start using 'mmes' and its auxiliary functions (e.g., 'vsm', 'usm', 'dsm', 'ism', etc.). This function will be no longer maintained.



iteration    LogLik     wall    cpu(sec)   restrained
    1      -33.5405   18:34:5      0           0
    2      -33.5405   18:34:5      0           0
    3      -33.5405   18:34:5      0           0
    4      -33.5405   18:34:5      0           0


Before we look at the results, let's break down the model statement

- `mmer()` is the mixed model solver used by sommer. There is also a `mmec()` function for solving mixed models that we will not cover here. For certain types of analysis 'mmec' will run faster.
- The first line of the model statement specifies the phenotype and then the fixed effects "block and variety.
- The second line gives the covariance structure for the residuals `rcov= ~units` is the default structure in which residuals are treated as being identically and indepenedently distributed. When using the default structure this line can be omitted, but there are many ways in which the residual covariance can be modeled and I prefer to always explicitly state the structure I'm using.
- The last line tells sommer which data frame to use and I set the verbose flag to TRUE so the **convergence** information is printed to the screen.

**Convergence** - Mixed models must be solved iteratively as you need estimates of the model effects to estimate variance composnents and you need the variance components to estimate the random effects. This means the solver must start by randomly initializing the variance components and interatively update the solutions until cpnvergence has been achieled. Mixed models do not always converge for a variety of reasons, so it is important to first check for model convergence before using and of the model outputs.

# Model Output

Now lets look at key component of the model output. First we will use the function `summary()` to look at what is contained in the object **model1**.

In [26]:
summary(model1)

This function has been deprecated. Please start using 'mmes' and its auxiliary functions (e.g., 'vsm', 'usm', 'dsm', 'ism', etc.). This function will be no longer maintained.



$groups
NULL

$varcomp
                           VarComp VarCompSE   Zratio Constraint
units.phenotype-phenotype 2.848182 0.4318402 6.595453   Positive

$betas
      Trait      Effect    Estimate Std.Error   t.value
1 phenotype (Intercept)  0.06147164 0.3081224  0.199504
2 phenotype        trt2  1.29941929 0.4357508  2.982024
3 phenotype        trt3 -0.86005383 0.4357508 -1.973729

$method
[1] "NR"

$logo
         logLik      AIC      BIC Method Converge
Value -33.54047 73.08094 80.58037     NR     TRUE

attr(,"class")
[1] "summary.mmer" "list"

**$groups** - Provides grouping information for more complicated covariance structures

**$varcomp** - provdes estimates of the variance components, in this case only for the residual variance component as there are no random effects in the model.

**$beta** - provides the estimates and standard errors of the fixed effects

**method** - Gives the method used to solve the mixed model equations `NR` is a second derivative method that will not be covered in this course.

**$logo** - Provides information on model fit and whether the model converged. Here we can see that Converged = TRUE, so we know the model converged successfuly. **Always confirm the model converged before using any results**.  

# Solutions

To pull out spcific components of **model1** we use `$`

The fixed effect solutions can be pulled out of the model object using the command `model1_ped$Beta`

In [27]:
sol_fixed=model1$Beta
sol_fixed

Trait,Effect,Estimate
<fct>,<fct>,<dbl>
phenotype,(Intercept),0.06147164
phenotype,trt2,1.29941929
phenotype,trt3,-0.86005383


Now I have my estimates from model 1, let's run a second model adds `exp_unit` as a random effect.

# Model 2 - Including `exp_unit` as a random effect

In [28]:
model2 <- mmer(phenotype ~ 1 + trt,
                 random = ~  exp_unit,
                 rcov=~ units,
                 data = dataf)

This function has been deprecated. Please start using 'mmes' and its auxiliary functions (e.g., 'vsm', 'usm', 'dsm', 'ism', etc.). This function will be no longer maintained.



iteration    LogLik     wall    cpu(sec)   restrained
    1      -26.8771   18:34:26      0           0
    2      -21.8612   18:34:26      0           0
    3      -20.5112   18:34:26      0           0
    4      -20.3965   18:34:26      0           0
    5      -20.3954   18:34:26      0           0
    6      -20.3954   18:34:26      0           0


As you can see the model statement is similar but we now have a new row for the random effect: `random= ~ exp_unit,`

`random= ~` tells the model that we are giving it terms we want to treat as random.


# Compare the output of model1 and model2. How do they differ? (1 pt)

In [ ]:
# code to compare the model output

# Answer:

# Run two additional models (2 pts)
## `model3` should have `exp_unit as a fixed effect
## `model4` should use the means of the complete dataset as the phenotype `datafM$phenotypeMean` only include trt as fixed and treat residuals as IID.

In [ ]:
# code for running the models here

# Compare the results of the 4 models. Do any of the models give the same results? Which model(s) do you think are giving the correct results and why?  (1 pt)

In [ ]:
# Code to compare the model output

# Answer

# Now repeat the 4 analyses but use the phenotype with missing data (`dataf$phenotypeMissing`) and phenotypic mean calculated using missing data (`datafM$phenotypeMeanMissing`). (4 pts)  

In [ ]:
# code to run the 4 models here

In [ ]:
# code to compare the results

# Compare the model output. Do any of the models give the same results? Which model do you think is best and why? (1 pt)

# Answer